In [ ]:
import pandas as pd
from ada_topics.config import DATA_DIR

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = False
pd.options.plotting.backend = "plotly"

In [ ]:
df = pd.read_feather(
    DATA_DIR / "piaac_computer_programming_by_country_and_age_group.arrow",
)

In [ ]:
print(df.to_html(border=0))

In [ ]:
df.groupby("age_group")

In [ ]:
print(df.groupby("age_group").std().to_html(border=0))

In [ ]:
df.groupby("country").max()

In [ ]:
df.groupby("country").quantile(0.5)

In [ ]:
grouped = df.reset_index().groupby("country")
grouped["age_group"].value_counts() / grouped["age_group"].count()